In [71]:
from glob import glob
from collections import defaultdict

import pandas as pd

import thicket as th

In [72]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# POINT THIS AT YOUR DATA

In [73]:
# Point this at the top directory of all your cali files for all of the implementations. Other files can be in the directory too, that is ok.
FILES_LOCATION = "./dataFiles/CUDA/Radix/random"

# Reader check
Can the files be read in one-by-one

In [74]:
working_files = []
error_files = []
i = 0
team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    try:
        tk = th.Thicket.from_caliperreader(file)
        working_files.append(file)
    except Exception:
        i += 1
        error_files.append(file)
    

In [75]:
print("Files that could be read in individually (one-by-one):")
print(f"{len(working_files)}/{len(team_files)} ({len(working_files)/len(team_files)*100}%)")

Files that could be read in individually (one-by-one):
35/35 (100.0%)


# Check for Metadata columns

Check for the necessary metadata columns from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#3b-collect-metadata)

In [76]:
team_metadata_valid = []
metadata_columns=['cali.caliper.version', 'spot.options', 'spot.channels', 'cali.channel',
    'launchdate',
    'libraries', 'cmdline', 'cluster', 'Algorithm', 'ProgrammingModel',
    'Datatype', 'SizeOfDatatype', 'InputSize', 'InputType',
    'group_num', 'implementation_source']
mpi_cols = ['num_procs']
cuda_cols = ['num_threads', 'num_blocks',]
metadata_col_dict = defaultdict(lambda: [])

team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    try:
        valid = True
        tk = th.Thicket.from_caliperreader(file)
        cols = tk.metadata.columns
        model_to_check = []
        if "CUDA" in tk.metadata["ProgrammingModel"].to_list()[0].upper():
            model_to_check = metadata_columns + cuda_cols
        else:
            model_to_check = metadata_columns + mpi_cols
        for col in model_to_check:
            if col not in cols:
                metadata_col_dict[list(tk.profile_mapping.values())[0]].append(col)
                valid=False
        if valid:
            team_metadata_valid.append(file)
    except KeyError:
        pass

for file, cols in metadata_col_dict.items():
    print(f"File '{file}' missing metadata columns:\n\t{cols}")

# Check for DataFrame columns

Check for the necessary DataFrame columns from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#4c-you-should-measure-the-following-performance-metrics). For the GPU columns, you need one or the other column in the tuple, not both.

In [62]:
team_dataframe_valid = []
necessary_columns = ["Min time/rank","Max time/rank","Avg time/rank","Total time",]
not_gpu_columns = ["Variance time/rank",]
gpu_columns=[("Avg GPU time/rank", "Avg GPU Time/rank"),
             ("Min GPU time/rank", "Min GPU Time/rank"),
             ("Max GPU time/rank", "Max GPU Time/rank"),
             ("Total GPU time", "Total GPU Time"),]
def check_df_cols(tk, dict):
    valid = True
    cols = tk.dataframe.columns
    for col in necessary_columns:
        if col not in cols:
            dict[list(tk.profile_mapping.values())[0]].append(col)
            valid = False
    if "ProgrammingModel" in tk.metadata.columns:
        if "CUDA" in tk.metadata["ProgrammingModel"].to_list()[0].upper():
            for col in gpu_columns:
                if col[0] not in cols and col[1] not in cols:
                    dict[list(tk.profile_mapping.values())[0]].append(col)
                    valid = False
        else:
            for col in not_gpu_columns:
                if col not in cols:
                    dict[list(tk.profile_mapping.values())[0]].append(col)
                    valid = False
    return valid

dataframe_col_dict = defaultdict(lambda: [])
team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    tk = th.Thicket.from_caliperreader(file)
    valid = check_df_cols(tk, dataframe_col_dict)
    if valid:
        team_dataframe_valid.append(file)

for file, cols in dataframe_col_dict.items():
    print(f"File '{file}' missing dataframe columns: {cols}")

# Try all files together

In [63]:
tk = th.Thicket.from_caliperreader(team_files)

# Check tree

Should be no different from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#3a-caliper-instrumentation), spelling and all.

In [64]:
tk.statsframe.dataframe["time"] = 1
print(tk.tree())

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2023.3.1

1.000 main
├─ 1.000 comm
│  └─ 1.000 comm_large
│     └─ 1.000 cudaMemcpy
├─ 1.000 comp
│  └─ 1.000 comp_large
├─ 1.000 correctness_check
└─ 1.000 data_init

Legend (Metric: time Min: 1.00 Max: 1.00)
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00

name User code    ◀  Only in left graph    ▶  Only in right graph



In [65]:
# Groupby programming model. Should result in 2 thickets, MPI and CUDA.
gb_pmodel = tk.groupby("ProgrammingModel")

2  thickets created...
{'CUDA': <thicket.thicket.Thicket object at 0x00000271CCA1C150>, nan: <thicket.thicket.Thicket object at 0x00000271CCE5D7D0>}


In [66]:
# Groupby the parameters we ran with. After this operation, each Thicket in gb_total should contain profiles with unique InputSizes (there should be no duplicate input sizes).
gb_cuda = gb_pmodel["CUDA"].groupby(["ProgrammingModel", "Algorithm", "InputType", "num_threads"])
# gb_mpi = gb_pmodel["MPI"].groupby(["ProgrammingModel", "Algorithm", "InputType", "num_procs"])
# gb_total = {**gb_cuda, **gb_mpi}

gb_total = {**gb_cuda}

3  thickets created...
{('CUDA', 'radix', 'Random', 64.0): <thicket.thicket.Thicket object at 0x00000271CCFB0510>, ('CUDA', 'radix', 'Random', 128.0): <thicket.thicket.Thicket object at 0x00000271CD4E8210>, ('CUDA', 'radix', 'Random', 256.0): <thicket.thicket.Thicket object at 0x00000271CCE8A890>}


In [67]:
# Compose all of the data back together. If this step errors, you probably have duplicate inputsizes. Run 1a to check for this.
ctk = th.Thicket.concat_thickets(
    thickets=list(gb_total.values()),
    axis="columns",
    headers=list(gb_total.keys()),
    metadata_key="InputSize"
)

In [68]:
ctk.dataframe.head(50)

(CUDA, radix, Random, 64.0)  \
                                                                                      nid   
node                                              InputSize                                 
{'name': 'main', 'type': 'function'}              65536.0                               1   
                                                  262144.0                              1   
                                                  1048576.0                             1   
                                                  4194304.0                             1   
                                                  16777216.0                            1   
                                                  67108864.0                            1   
                                                  268435456.0                           1   
{'name': 'comm', 'type': 'function'}              65536.0                               3   
                                                  262144.0                              3   
                                                  1048576.0                             3   
                                                  4194304.0                             3   
                                                  16777216.0                            3   
                                                  67108864.0                            3   
                                                  268435456.0                           3   
{'name': 'comm_large', 'type': 'function'}        65536.0                               4   
                                                  262144.0                              4   
                                                  1048576.0                             4   
                                                  4194304.0                             4   
                                                  16777216.0                            4   
                                                  67108864.0                            4   
                                                  268435456.0                           4   
{'name': 'cudaMemcpy', 'type': 'function'}        65536.0                               5   
                                                  262144.0                              5   
                                                  1048576.0                             5   
                                                  4194304.0                             5   
                                                  16777216.0                            5   
                                                  67108864.0                            5   
                                                  268435456.0                           5   
{'name': 'comp', 'type': 'function'}              65536.0                               6   
                                                  262144.0                              6   
                                                  1048576.0                             6   
                                                  4194304.0                             6   
                                                  16777216.0                            6   
                                                  67108864.0                            6   
                                                  268435456.0                           6   
{'name': 'comp_large', 'type': 'function'}        65536.0                               7   
                                                  262144.0                              7   
                                                  1048576.0                             7   
                                                  4194304.0                             7   
                                                  16777216.0                            7   
                                                  67108864.0   

# 1A

Check for duplicate input sizes

In [69]:
i = 0
for key in list(gb_total.keys()):
    print(i)
    print(gb_total[key].profile_mapping)
    print(gb_total[key].metadata["InputSize"])
    i += 1

0
OrderedDict([(74918312, './dataFiles/CUDA/Radix/random\\64-1048576.cali'), (369754120, './dataFiles/CUDA/Radix/random\\64-16777216.cali'), (1838247881, './dataFiles/CUDA/Radix/random\\64-65536.cali'), (1954005080, './dataFiles/CUDA/Radix/random\\64-268435456.cali'), (2237570457, './dataFiles/CUDA/Radix/random\\64-67108864.cali'), (3128975555, './dataFiles/CUDA/Radix/random\\64-262144.cali'), (3963780357, './dataFiles/CUDA/Radix/random\\64-4194304.cali')])
profile
74918312        1048576.0
369754120      16777216.0
1838247881        65536.0
1954005080    268435456.0
2237570457     67108864.0
3128975555       262144.0
3963780357      4194304.0
Name: InputSize, dtype: float64
1
OrderedDict([(160990904, './dataFiles/CUDA/Radix/random\\128-65536.cali'), (746988520, './dataFiles/CUDA/Radix/random\\128-268435456.cali'), (1879721705, './dataFiles/CUDA/Radix/random\\128-1048576.cali'), (2133676039, './dataFiles/CUDA/Radix/random\\128-262144.cali'), (2133879534, './dataFiles/CUDA/Radix/random\